In [31]:
from surprise import NMF, SVD, SVDpp, KNNBasic, KNNWithMeans, KNNWithZScore, CoClustering
from surprise.model_selection import cross_validate
from surprise import Reader, Dataset

import pandas as pd
import numpy as np



In [3]:
import pandas as pd

movies = pd.read_csv('movies.csv')
rating = pd.read_csv('ratings.csv')

In [22]:
movie_names=movies[['movie_id','title']]
combined_movies_data = pd.merge(rating, movie_names, on='movie_id')
combined_movies_data = combined_movies_data[['user_id','title', 'rating']]


In [68]:
rating

,Unnamed: 0,user_id,movie_id,rating,timestamp,user_emb_id,movie_emb_id
0,0,1,1193,5,978300760,0,1192
1,1,1,661,3,978302109,0,660
2,2,1,914,3,978301968,0,913
3,3,1,3408,4,978300275,0,3407
4,4,1,2355,5,978824291,0,2354
...,...,...,...,...,...,...,...
1000204,1000204,6040,1091,1,956716541,6039,1090
1000205,1000205,6040,1094,5,956704887,6039,1093
1000206,1000206,6040,562,5,956704746,6039,561
1000207,1000207,6040,1096,4,956715648,6039,1095


In [97]:
#assume Sally's user id is 1001

combined_movies_data[combined_movies_data ["user_id"]==1002]

,user_id,title,rating
23670,1002,Back to the Future (1985),3
38302,1002,"Sixth Sense, The (1999)",3
40580,1002,Apollo 13 (1995),5
43926,1002,Rain Man (1988),5
52702,1002,Saving Private Ryan (1998),4
...,...,...,...
829905,1002,"Right Stuff, The (1983)",5
916078,1002,Wonder Boys (2000),4
916538,1002,"Horse Whisperer, The (1998)",2
937893,1002,Local Hero (1983),4


In [37]:

# rename the columns to userID, itemID and rating
combined_movies_data.columns = ['user_id', 'title', 'rating']

# use the transform method group by userID and count to keep the movies with more than 25 reviews

combined_movies_data['reviews'] = combined_movies_data.groupby(['title'])['rating'].transform('count')

combined_movies_data= combined_movies_data[combined_movies_data.reviews>25][['user_id', 'title','rating']]

In [41]:
combined_movies_data

,user_id,title,rating
0,1,One Flew Over the Cuckoo's Nest (1975),5
1,2,One Flew Over the Cuckoo's Nest (1975),5
2,12,One Flew Over the Cuckoo's Nest (1975),4
3,15,One Flew Over the Cuckoo's Nest (1975),4
4,17,One Flew Over the Cuckoo's Nest (1975),5
...,...,...,...
999110,5548,Manny & Lo (1996),3
999111,5604,Manny & Lo (1996),4
999112,5855,Manny & Lo (1996),4
999113,5925,Manny & Lo (1996),4


In [38]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(combined_movies_data, reader)

In [55]:
data

In [47]:
#assume the user is Sally, 1001; Linda, 1005

user=1002

In [48]:
# get the list of the movie ids
unique_ids = combined_movies_data['title'].unique()

# get the list of the ids that the userid 1001 has rated
iids_user = combined_movies_data.loc[combined_movies_data['user_id']==user, 'user_id']

# remove the rated movies for the recommendations
movies_to_predict = np.setdiff1d(unique_ids,iids_user)

In [ ]:
my_recs

In [72]:
#recommedation according to NMF

algo = NMF()
algo.fit(data.build_full_trainset())

my_recs = []
for iid in movies_to_predict:
    my_recs.append((iid, algo.predict(uid=user,iid=iid).est))
    
pd.DataFrame(my_recs, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)

,iid,predictions
1457,"King of Masks, The (Bian Lian) (1996)",5.0
1999,Paths of Glory (1957),5.0
1933,On the Waterfront (1954),5.0
1941,One Flew Over the Cuckoo's Nest (1975),5.0
996,Fresh (1994),5.0
1980,"Palm Beach Story, The (1942)",5.0
1987,Paradise Lost: The Child Murders at Robin Hood...,5.0
959,For All Mankind (1989),5.0
1998,Pather Panchali (1955),5.0
2004,"Pawnbroker, The (1965)",5.0


In [88]:
algo

In [71]:
#recommedation according to svd

algo = SVD()
algo.fit(data.build_full_trainset())

my_recs = []
for iid in movies_to_predict:
    my_recs.append((iid, algo.predict(uid=user,iid=iid).est))
    


In [73]:
pd.DataFrame(my_recs, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)

,iid,predictions
1457,"King of Masks, The (Bian Lian) (1996)",5.0
1999,Paths of Glory (1957),5.0
1933,On the Waterfront (1954),5.0
1941,One Flew Over the Cuckoo's Nest (1975),5.0
996,Fresh (1994),5.0
1980,"Palm Beach Story, The (1942)",5.0
1987,Paradise Lost: The Child Murders at Robin Hood...,5.0
959,For All Mankind (1989),5.0
1998,Pather Panchali (1955),5.0
2004,"Pawnbroker, The (1965)",5.0


3.5854122193466105

In [99]:
user=1002
iid=23670

my_recs = []
for iid in movies_to_predict:
    my_recs.append((iid, algo.predict(uid=user,iid=iid).est))
    
pd.DataFrame(my_recs, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)

,iid,predictions
104,American Dream (1990),5.000000
1446,"Killing Fields, The (1984)",5.000000
297,"Bicycle Thief, The (Ladri di biciclette) (1948)",5.000000
21,42 Up (1998),5.000000
959,For All Mankind (1989),5.000000
523,Chinatown (1974),5.000000
1122,"Great Day in Harlem, A (1994)",4.996118
2757,"Usual Suspects, The (1995)",4.978957
154,Apocalypse Now (1979),4.970191
2791,Wallace & Gromit: The Best of Aardman Animatio...,4.966130


In [91]:
algo

In [56]:
#recommedation according to svd++; too heavy to run! not recommeded

algo = SVDpp()
algo.fit(data.build_full_trainset())

my_recs = []
for iid in movies_to_predict:
    my_recs.append((iid, algo.predict(uid=user,iid=iid).est))
    
pd.DataFrame(my_recs, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)

KeyboardInterrupt: 

In [57]:
#recommedation according to KNN with Z-Score

algo = KNNWithZScore()
algo.fit(data.build_full_trainset())

my_recs = []
for iid in movies_to_predict:
    my_recs.append((iid, algo.predict(uid=1001,iid=iid).est))
    
pd.DataFrame(my_recs, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)

Computing the msd similarity matrix...


KeyboardInterrupt: 

In [ ]:
#recommedation according to CoClustering

algo = CoClustering()
algo.fit(data.build_full_trainset())

my_recs = []
for iid in movies_to_predict:
    my_recs.append((iid, algo.predict(uid=1001,iid=iid).est))
    
pd.DataFrame(my_recs, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)

In [110]:
algo.preidct(data.build_full_trainset())


AttributeError: 'NMF' object has no attribute 'preidct'

In [103]:
reader = Reader(rating_scale=(1, 5))
subset = Dataset.load_from_df(combined_movies_data.sample(n=100000), reader)

In [104]:

cv = []
# Iterate over all recommender system algorithms
for recsys in [NMF(), SVD(),SVDpp(), KNNWithZScore(), CoClustering()]:
    # Perform cross validation
    tmp = cross_validate(recsys, subset, measures=['RMSE'], cv=3, verbose=False)
    cv.append((str(recsys).split(' ')[0].split('.')[-1], tmp['test_rmse'].mean()))

pd.DataFrame(cv, columns=['RecSys', 'RMSE'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,RecSys,RMSE
0,NMF,1.058736
1,SVD,0.953635
2,SVDpp,0.949966
3,KNNWithZScore,1.032214
4,CoClustering,1.017598
